# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe2d4ac6c40>
├── 'a' --> tensor([[-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743]])
└── 'x' --> <FastTreeValue 0x7fe2d4ac6e50>
    └── 'c' --> tensor([[-0.1092,  0.2905, -0.4219,  0.7781],
                        [ 0.8802, -0.4592, -1.2742, -0.8595],
                        [ 0.5969, -1.3623,  0.2230, -0.4164]])

In [4]:
t.a

tensor([[-2.1000,  0.5904,  0.5108],
        [ 0.7393, -1.1363,  0.0743]])

In [5]:
%timeit t.a

66 ns ± 0.0962 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe2d4ac6c40>
├── 'a' --> tensor([[-0.6924,  0.1709,  0.2756],
│                   [-1.0050,  1.1657,  0.4360]])
└── 'x' --> <FastTreeValue 0x7fe2d4ac6e50>
    └── 'c' --> tensor([[-0.1092,  0.2905, -0.4219,  0.7781],
                        [ 0.8802, -0.4592, -1.2742, -0.8595],
                        [ 0.5969, -1.3623,  0.2230, -0.4164]])

In [7]:
%timeit t.a = new_value

67.7 ns ± 0.0786 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743]]),
    x: Batch(
           c: tensor([[-0.1092,  0.2905, -0.4219,  0.7781],
                      [ 0.8802, -0.4592, -1.2742, -0.8595],
                      [ 0.5969, -1.3623,  0.2230, -0.4164]]),
       ),
)

In [10]:
b.a

tensor([[-2.1000,  0.5904,  0.5108],
        [ 0.7393, -1.1363,  0.0743]])

In [11]:
%timeit b.a

54.8 ns ± 0.0293 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.1293, -0.4173,  0.6356],
               [-1.6237,  2.1675, -0.6240]]),
    x: Batch(
           c: tensor([[-0.1092,  0.2905, -0.4219,  0.7781],
                      [ 0.8802, -0.4592, -1.2742, -0.8595],
                      [ 0.5969, -1.3623,  0.2230, -0.4164]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.351 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

882 ns ± 8.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 55.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 577 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe1f5c1e700>
├── 'a' --> tensor([[[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]],
│           
│                   [[-2.1000,  0.5904,  0.5108],
│                    [ 0.7393, -1.1363,  0.0743]]])
└── 'x' --> <FastTreeValue 0x7fe2d7af1c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe2d4a6fc70>
├── 'a' --> tensor([[-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743],
│                   [-2.1000,  0.5904,  0.5108],
│                   [ 0.7393, -1.1363,  0.0743]])
└── 'x' --> <FastTreeValue 0x7fe2d4a6fd90>
    └── 'c' --> tensor([[-0.1092,  0.2905, -0.4219,  0.7781],
                        [ 0.8802, -0.4592, -1.2742, -0.8595],
                 

In [23]:
%timeit t_cat(trees)

31.4 µs ± 77.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.5 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]],
       
               [[-2.1000,  0.5904,  0.5108],
                [ 0.7393, -1.1363,  0.0743]]]),
    x: Batch(
           c: tensor([[[-0.1092,  0.2905, -0.4219,  0.7781],
                       [ 0.8802, -0.4592, -1.2742, -0.8595],
                       [ 0.5969, -1.3623,  0.2230, -0.4164]],
         

In [26]:
%timeit Batch.stack(batches)

80.8 µs ± 95 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743],
               [-2.1000,  0.5904,  0.5108],
               [ 0.7393, -1.1363,  0.0743]]),
    x: Batch(
           c: tensor([[-0.1092,  0.2905, -0.4219,  0.7781],
                      [ 0.8802, -0.4592, -1.2742, -0.8595],
                      [ 0.5969, -1.3623,  0.2230, -0.4164],
                      [-0.1092,  0.2905, -0.4219,  0.7781],
                      [ 0.8802, -

In [28]:
%timeit Batch.cat(batches)

152 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

363 µs ± 8.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
